In [1]:
import phyloGraph as ph

from plotly.offline import download_plotlyjs, init_notebook_mode
init_notebook_mode(connected=True)

from time import time
import numpy as np
import pandas as pd

#vim ~/.Plotly/.Credentials

## Load Data

In [2]:
pgd = ph.phyloData()

the data of phylo


Pick one of the methods below for getting data

### 1) fetch from Tree of Life

In [3]:
pgd.search_name("primates")

,name,id
0,Primates,15963


In [4]:
#pgd.fetch_tol_data(15963, limit=25000)

In [5]:
#pgd.add_time()
#pgd.df.head()

### 2) load from raw file

In [6]:
#raw_file = "data/primate_raw.xml"
#pgd.load_raw_file(raw_file)

In [7]:
#pgd.write_prep_data("data/mammal_df.csv", "data/mammal_links.txt")

### 3) load from prepared files

In [8]:
pgd.load_prep_data("data/Mammalia-15040-df-fixedage.csv", 15040)
#pgd.load_prep_data("data/Chordata-2499-df-fixedage.csv", 2499)


### return data for plotting

In [9]:
start = time()
pgd.jitter_XY()
print("jitter_XY() took {} secs".format(np.round(time() - start)))

jittering (all=False)
jitter_XY() took 23.0 secs


## Plot Data

In [10]:
pgp = ph.phyloGraph(pgd)

the plotting of phylo


In [11]:
#root_node=14829 # Vertebrata
root_node=15040 # mammals
#root_node=15963 # primates

In [12]:
start = time()
pgp.create_plot_df(root_node)
print("create_plot_df() took {} secs".format(np.round(time() - start)))

create_plot_df() took 2.0 secs


In [13]:
#pgp.plot_df.to_csv("data/plot_df-Vertebrata.csv", index=False)

In [14]:
start = time()
pgp.create_plot_data()
#pgp.create_plot_data(Z_dim='depth', Z_dim_mult=-1)
print("create_plot_data() took {} secs".format(np.round(time() - start)))

Loaded plot data. Root node: 15040
create_plot_data() took 4.0 secs


In [15]:
#pgp.render_plot()

In [16]:
pgp.search_name('Euther')

        name     id
19  Eutheria  15997


In [17]:
#focus_node = 16024 # whales
#focus_node = 16293 # Catarrhini
#focus_node = 16299 # Hominidae
focus_node = 15991 # monotremes
pgp.refocus(focus_node)

/Users/Seth/Documents/tolGraph/phyloGraph.py:895: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Loaded plot data. Highlighting: 15991
